In [17]:
# Import Dependencies
import pandas as pd
import numpy as np

In [18]:
#Import zipcode key csv file
zip_code_csv = "../Resources/zip_code_database.csv"

In [19]:
#Create Original DataFrame
zip_code_df = pd.read_csv(zip_code_csv)
zip_code_df = zip_code_df.replace(np.nan, '', regex=True)

In [30]:
#Filter on just Colorado Zip Codes and sort by population, take top 25 most populated zips in CO
co_zip_code = zip_code_df.loc[(zip_code_df["state"] == "CO"),:]
co_zip_code = co_zip_code.sort_values(by = "irs_estimated_population_2015", ascending=False)
co_zip_code = co_zip_code.nlargest(50, "irs_estimated_population_2015", keep='first')
co_zip_code = co_zip_code[["zip","latitude","longitude"]]
co_zip_code = co_zip_code.set_index("zip")
co_zip_code

,latitude,longitude
zip,,
80013,39.66,-104.77
80015,39.63,-104.78
80219,39.70,-105.04
80134,39.50,-104.76
80634,40.42,-104.74
80011,39.74,-104.78
80016,39.60,-104.71
80504,40.16,-105.03
80229,39.86,-104.96
